# Importación de librerias requeridas

In [47]:
#!pip install torchsummary

In [48]:
# -*- coding: utf-8 -*-
from __future__ import print_function, division

# ===========================
# CONFIGURACIÓN INICIAL
# ===========================

# tqdm es una librería para mostrar barras de progreso en ciclos (loops).
# "tqdm.auto" detecta si estamos en un notebook (como Kaggle o Jupyter)
# o en una terminal, y se adapta automáticamente sin mostrar advertencias.
from tqdm.auto import tqdm
tqdm.pandas()   # Integra tqdm con pandas → se ven barras de progreso en operaciones de pandas.

# ===========================
# MANEJO DE WEIGHTS & BIASES (wandb)
# ===========================
# wandb es una herramienta para registrar experimentos de machine learning.
# En Kaggle a veces genera errores o no queremos usarlo.
# Con esta configuración lo desactivamos por defecto y creamos un "plan B"
# para que el código siga funcionando aunque wandb falle o no esté instalado.

import os
os.environ.setdefault("WANDB_DISABLED", "true")  # Kaggle lo desactiva automáticamente

try:
    import wandb  # Intentamos importar wandb
except Exception as e:
    # Si falla la importación, creamos una clase de "simulación"
    # que actúa como reemplazo básico (stub).
    # Así, cuando en el código se llame a wandb.init() o wandb.log(),
    # no se producirá un error.
    class _WandbStub:
        def init(self, *args, **kwargs):
            class _Ctx:
                def __enter__(self): return self
                def __exit__(self, exc_type, exc, tb): pass
            return _Ctx()
        def log(self, *args, **kwargs): pass
        def watch(self, *args, **kwargs): pass
        def finish(self, *args, **kwargs): pass

    wandb = _WandbStub()  # Reemplazamos wandb por el "stub"

# ===========================
# IMPORTACIÓN DE LIBRERÍAS
# ===========================
# Estas librerías cubren diferentes tareas:
# - Numpy/Pandas: análisis de datos
# - PyTorch/Torchvision: redes neuronales
# - Albumentations: aumentación de imágenes
# - Scikit-learn: partición de datos
# - OpenCV/Skimage: procesamiento de imágenes
# - Matplotlib: visualización

import numpy as np
import pandas as pd
from numpy.typing import NDArray
from functools import reduce
from itertools import islice, chain
import math, copy

from PIL import Image

import torch
from torch import nn, Tensor
from torch.optim import Optimizer
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchsummary import summary  # Muestra la arquitectura de la red

# Albumentations: librería de aumentación de datos para imágenes
import albumentations as A

from sklearn.model_selection import train_test_split
from multiprocessing import cpu_count  # Para paralelizar procesos

import os.path as osp
from skimage import io, transform
import matplotlib.pyplot as plt
import typing as ty
import cv2

plt.ion()  # Activa el "modo interactivo" → las gráficas se actualizan automáticamente.

# ===========================
# EXPLORACIÓN DE DATOS EN KAGGLE
# ===========================
# Kaggle guarda los datasets en la carpeta "/kaggle/input".
# El siguiente bloque recorre esa carpeta y muestra los primeros 10 archivos encontrados.
# Esto ayuda a verificar qué datos tenemos disponibles sin abrir manualmente el explorador.
for root, dirs, filenames in os.walk('/kaggle/input'):
    for i, filepath in enumerate(filenames):
        if i >= 10:  # Solo mostramos hasta 10 archivos para no saturar la salida
            print()
            break
        print(osp.join(root, filepath))


/kaggle/input/aa-iv-2025-ii-object-localization/sample_submission.csv
/kaggle/input/aa-iv-2025-ii-object-localization/train.csv
/kaggle/input/aa-iv-2025-ii-object-localization/test.csv
/kaggle/input/aa-iv-2025-ii-object-localization/images/videoplayback-1-_mp4-6_jpg.rf.e2195c50e4aa68ffc18f41c80fd7d235.jpg
/kaggle/input/aa-iv-2025-ii-object-localization/images/IMG_4861_mp4-43_jpg.rf.4271f075e21c21ee8dc01731c6a7ea89.jpg
/kaggle/input/aa-iv-2025-ii-object-localization/images/IMG_4861_mp4-56_jpg.rf.2c4bd7a2dd787d0344f2b49af88f21f1.jpg
/kaggle/input/aa-iv-2025-ii-object-localization/images/IMG_3100_mp4-24_jpg.rf.894c40eafa77cd73ff50a69982e3f924.jpg
/kaggle/input/aa-iv-2025-ii-object-localization/images/IMG_4860_mp4-45_jpg.rf.29bb394fd84df979b2a6096746751f42.jpg
/kaggle/input/aa-iv-2025-ii-object-localization/images/IMG_4861_mp4-9_jpg.rf.bcc352b97426c7378bcd8004247f4433.jpg
/kaggle/input/aa-iv-2025-ii-object-localization/images/video_CDC-YOUTUBE_mp4-41_jpg.rf.4f56be4b40c9775509474d515489f5a5

## **Definición de Funciones y Clases**

-   **Visualización de bounding boxes**: Funciones para dibujar cajas delimitadoras y etiquetas de clase sobre imágenes, usando OpenCV.
-   **Dataset personalizado**:  `maskDataset`  extiende  `torch.utils.data.Dataset`  para manejar imágenes, bounding boxes y clases, permitiendo transformaciones y resize.
-   **Transformaciones**: Wrappers para convertir imágenes a tensores (`ToTensor`), normalizar canales (`Normalizer`), y aplicar augmentations con Albumentations o torchvision.
-   **Feature extractor**:  `ResNetFeatureExtractor`  encapsula un modelo ResNet, eliminando la capa final y añadiendo flatten y dropout para extraer embeddings.
-   **Métricas y pérdidas**: Funciones para calcular IoU, accuracy y una función de pérdida multitarea (clasificación + regresión de bbox).
-   **Entrenamiento y evaluación**: Funciones para el ciclo de entrenamiento, evaluación periódica y logging de métricas.

## DEFINICION DE ARQUITECTURA CNN

El Siguiente fragmento de código define las dos "cabezas" principales del modelo multitarea para detección de objetos: la cabeza de clasificación y la cabeza de regresión de bounding box. Ambas están implementadas como secuencias de capas densas (fully connected) con normalización, activaciones y dropout para mejorar la capacidad de generalización y la estabilidad durante el entrenamiento.

La  **cabeza de clasificación**  (`cls_head`) recibe el vector de características extraído por el backbone (ResNet50_2) y lo transforma a través de varias capas lineales. Primero, reduce la dimensionalidad a 768 unidades, aplica normalización por lotes (BatchNorm1d), una activación SiLU y dropout para evitar el sobreajuste. Este patrón se repite con una reducción a 256 unidades y un dropout más alto, finalizando con una capa lineal que produce los logits para cada clase.

Se utilizaron valores elevados de dropout en la cabeza de clasificación porque esta alcanzaba una alta precisión en pocas iteraciones, lo que indicaba un riesgo de sobreajuste. Al incrementar el dropout, se ralentiza el aprendizaje de la clasificación, permitiendo que la cabeza de regresión tenga más tiempo para optimizarse  Así, se mejora la generalización del modelo y se evita que la clasificación domine el proceso de aprendizaje antes de que la regresión converja adecuadamente.

Adicionalmente, se dio mayor peso a la tarea de regresión en la función de pérdida, utilizando un parámetro alpha de 0.7, y se asignaron learning rates diferenciados para el backbone, la cabeza de clasificación y la cabeza de regresión, favoreciendo un ajuste más fino y específico para cada componente del modelo.


La función de activación **SiLU** (Sigmoid Linear Unit), se utiliza en vez de **ReLU** porque ofrece varias ventajas en redes profundas y tareas de visión por computadora. SiLU es una función suave, esto es el resultado de experimentacion usando diferentes funciones de activacion, con Relu se obtivieron resultados mas bajos


La  **cabeza de regresión**  (`reg_head`) está diseñada para predecir las coordenadas de bbox. Comienza disminuyendo el vector de entrada a la mitad (1024 unidades), seguido de normalización, activación ReLU y dropout. Luego, reduce progresivamente la dimensionalidad a 512 y 256 unidades, manteniendo la normalización y la activación, y finalmente produce cuatro valores que corresponden a las coordenadas normalizadas  `[xmin, ymin, xmax, ymax]`. 


La cantidad de capas en la arquitectura se definió como resultado de diferentes experimentos y comparación de resultados, buscando el mejor equilibrio entre capacidad de representación y generalización del modelo.

En la función de pérdida, además del alpha que pondera la contribución entre la tarea de clasificación y regresión, se asignaron pesos [0.35, 0.65] a las clases con el fin de mitigar el desbalance en la distribución de etiquetas y favorecer el aprendizaje de la clase minoritaria. 

In [49]:
def get_output_shape(model: nn.Module, image_dim: ty.Tuple[int, int, int]):
    # ===========================================================
    # UTILIDAD: INFERIR LA FORMA DE SALIDA DEL BACKBONE
    # -----------------------------------------------------------
    # - Recibe un 'model' (ej. nuestro extractor VGG16) ya movido a 'device'.
    # - 'image_dim' debe incluir el batch (B, C, H, W). En el uso típico abajo
    #   se pasa como [1, *input_shape] → [1, 3, 640, 640].
    # - Crea un tensor aleatorio con esa forma, lo pasa por el modelo
    #   y devuelve la 'shape' resultante (solo para inspección).
    # ===========================================================
    return model(torch.rand(*(image_dim)).to(device)).data.shape


class Model(nn.Module):
    def __init__(self, input_shape: ty.Tuple[int, int, int] = (3, 256, 256), n_classes: int = 2):
        """
        MODELO MULTI-TAREA (CLASIFICACIÓN + REGRESIÓN DE BBOX)

        Entradas
        --------
        input_shape : (C, H, W)
            Tamaño esperado de la imagen (usamos 3×640×640).
            Formato PyTorch: canal primero (C,H,W).
        n_classes : int
            Nº de clases para clasificación (en este curso: 2 → no-mask / mask).

        Descripción
        -----------
        El modelo tiene:
          • Un 'backbone' convolucional (preentrenado) que extrae un vector de
            características (features) a partir de la imagen.
          • Una cabeza de CLASIFICACIÓN (cls_head) que toma ese vector y produce
            'logits' por clase (forma [B, n_classes]).
          • Una cabeza de REGRESIÓN (reg_head) que toma el mismo vector y predice
            las 4 coordenadas de la caja (forma [B, 4]).
        """
        super().__init__()

        self.input_shape = input_shape
        # Backbone preentrenado 
        self.backbone = pretrained_model_resnet

        # Inferimos cuántas características (F) salen del backbone para este input.
        # Se usa un batch sintético de 1 imagen: [1, C, H, W].
        backbone_output_shape = get_output_shape(self.backbone, [1, *input_shape])
        # Aplastamos todas las dimensiones de salida para obtener F (nº de features).
        backbone_output_features = reduce(lambda x, y: x*y, backbone_output_shape)

        # ---------------------------
        # CABEZA DE CLASIFICACIÓN 
        # ---------------------------
        # Toma el vector de features (F) y produce 'logits' de tamaño n_classes.
        self.cls_head = nn.Sequential(
            nn.Linear(backbone_output_features, 768),
            nn.BatchNorm1d(768),
            nn.SiLU(),
            nn.Dropout(0.5),

            nn.Linear(768, 256),
            nn.BatchNorm1d(256),
            nn.SiLU(),
            nn.Dropout(0.6),

            nn.Linear(256, n_classes)  # logits
        )

        # ---------------------------
        # CABEZA DE REGRESIÓN (BBOX)
        # ---------------------------
        # Predice 4 valores: [xmin, ymin, xmax, ymax] en la MISMA escala que tus etiquetas.
        self.reg_head = nn.Sequential(
            nn.Linear(backbone_output_features, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),

            nn.Linear(256, 4)
        )

    def forward(self, x: Tensor) -> ty.Dict[str, Tensor]:
        # ===========================================================
        # FLUJO HACIA ADELANTE
        # x: tensor de imágenes [B, 3, 640, 640]
        # 1) Extraemos features con el backbone → [B, F]
        # 2) cls_head(features) → logits de clase [B, n_classes]
        # 3) reg_head(features) → bbox [B, 4]
        # 4) Devolvemos un diccionario con ambas salidas
        # ===========================================================
        features = self.backbone(x)
        cls_logits = self.cls_head(features)
        pred_bbox = self.reg_head(features)
        predictions = {'bbox': pred_bbox, 'class_id': cls_logits}
        return predictions


In [50]:

# ---------------------------------------------------------------------------
# FIRMAS DE TRANSFORMACIONES (TIPADO)
# ---------------------------------------------------------------------------
# Cada transformación debe recibir y devolver un diccionario de numpy arrays
# con claves como 'image', 'bbox' y/o 'class_id'. Esto ayuda a documentar
# la interfaz esperada por el pipeline de datos.
transform_func_inp_signature = ty.Dict[str, NDArray[np.float64]]
transform_func_signature = ty.Callable[
    [transform_func_inp_signature],  # entrada: sample dict
    transform_func_inp_signature     # salida: sample dict (misma estructura)
]


# ===========================
# VISUALIZACIÓN Y BBOX
# ===========================
def draw_bbox(img, bbox, color,thickness: int = 3):
    # Dibuja un único cuadro delimitador (bounding box) sobre una imagen.
    xmin, ymin, xmax, ymax = bbox
    img = cv2.rectangle(img, (xmin, ymin), (xmax, ymax), color, thickness)
    return img

def normalize_bbox(bbox, h: int, w: int):
    """Escala las coordenadas normalizadas al tamaño real de la imagen."""
    return [
        int(bbox[0] * w),  # xmin
        int(bbox[1] * h),  # ymin
        int(bbox[2] * w),  # xmax
        int(bbox[3] * h),  # ymax,
    ]

def draw_bboxes(imgs, bboxes, colors,thickness):
    """Dibuja múltiples cuadros delimitadores en imágenes, escalando según h y w."""
    for i, (img, bbox, color) in enumerate(zip(imgs, bboxes, colors)):
        imgs[i] = draw_bbox(img, bbox, color,thickness)
    return imgs

def draw_classes(imgs, classes, colors, origin, prefix: str ='',fontScale : int = 2):
    """Dibuja las clases en las imágenes."""
    for i, (img, class_id, color) in enumerate(zip(imgs, classes, colors)):
        if type(c)==list:
            name_class_=id2obj[classes[i]]
        else:
            name_class_=id2obj[classes[i][0]]
        imgs[i] = cv2.putText(
            img, f'{prefix}{name_class_}',
            origin, cv2.FONT_HERSHEY_SIMPLEX,
            fontScale , color, 2, cv2.LINE_AA
        )
    return imgs

def draw_predictions(imgs, classes, bboxes, colors, origin,thickness,fontScale):
    """Combina las funciones anteriores para dibujar cuadros delimitadores y clases en las imágenes."""
    assert all(len(x) > 0 for x in [imgs, classes, bboxes, colors])
    if len(colors) == 1:
        colors = [colors[0] for _ in imgs]
    imgs = draw_bboxes(imgs, bboxes, colors,thickness)
    imgs = draw_classes(imgs, classes, colors, origin,"",fontScale)
    return imgs

# ===========================
# DATASET Y MANEJO DE DATOS
# ===========================

def extract_any_int(name: str) -> int:
    """Extrae el último grupo de dígitos de un nombre de archivo."""
    base, _ = os.path.splitext(name)
    nums = re.findall(r'\d+', base)
    return int(nums[-1]) if nums else -1

class maskDataset(Dataset):
    """Dataset personalizado para imágenes y bounding boxes."""
    def __init__(
        self,
        df: pd.DataFrame,
        root_dir: str,
        labeled: bool = True,
        transform: ty.Optional[ty.List[transform_func_signature]] = None,
        output_size: ty.Optional[tuple] = None
    ) -> None:
        self.df = df
        self.root_dir = root_dir
        self.transform = transform
        self.labeled = labeled
        self.output_size = output_size

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx: int) -> transform_func_signature:
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = os.path.join(self.root_dir, self.df.filename.iloc[idx])
        image = io.imread(img_name)
        if image is None:
            raise FileNotFoundError(f"Image not found: {img_name}")
        if image.ndim == 2:
            image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        elif image.shape[2] == 4:
            image = image[:, :, :3]
        if self.output_size:
            image = cv2.resize(image, self.output_size)
        sample = {'image': image}
        if self.labeled:
            img_class = self.df.class_id.iloc[idx]
            img_bbox = self.df.iloc[idx, 1:5]
            img_bbox = np.array([img_bbox]).astype('float')
            img_class = np.array([img_class]).astype('int')
            sample.update({'bbox': img_bbox, 'class_id': img_class})
        if self.transform:
            sample = self.transform(sample)
        return sample

# ===========================
# TRANSFORMACIONES
# ===========================

class ToTensor(object):
    """Convierte ndarrays en sample a Tensors."""
    def __call__(self, sample):
        image = sample['image']
        image = image.transpose((2, 0, 1))
        image = torch.from_numpy(image).float()
        sample.update({'image': image})
        return sample

class Normalizer(object):
    """Normaliza la imagen por canal usando mean y std."""
    def __init__(self, stds, means):
        self.stds = stds
        self.means = means
    def __call__(self, sample):
        image = sample['image']
        for channel in range(3):
            image[channel] = (image[channel] - means[channel]) / stds[channel]
        sample['image'] = image
        return sample

class TVTransformWrapper(object):
    """Wrapper para transforms de torchvision."""
    def __init__(self, transform: torch.nn.Module):
        self.transform = transform
    def __call__(self, sample):
        sample['image'] = self.transform(sample['image'])
        return sample

class AlbumentationsWrapper(object):
    """Wrapper para transforms de Albumentations."""
    def __init__(self, transform):
        self.transform = transform
    def __call__(self, sample):
        transformed = self.transform(
            image=sample['image'],
            bboxes=sample['bbox'],
        )
        sample['image'] = transformed['image']
        sample['bbox'] = np.array(transformed['bboxes'])
        return sample

# ===========================
# ARQUITECTURA Y UTILIDADES DE MODELO
# ===========================

def get_output_shape(model: nn.Module, image_dim: ty.Tuple[int, int, int]):
    """Inferir la forma de salida del backbone."""
    return model(torch.rand(*(image_dim)).to(device)).data.shape

class ResNetFeatureExtractor(nn.Module):
    """Extractor de features basado en ResNet."""
    def __init__(self, model):
        super(ResNetFeatureExtractor, self).__init__()
        self.features = nn.Sequential(*list(model.children())[:-1])
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.7)
    def forward(self, x):
        out = self.features(x)
        out = self.flatten(out)
        out = self.dropout(out)
        return out


# ===========================
# MÉTRICAS Y PÉRDIDAS
# ===========================

def iou(y_true: Tensor, y_pred: Tensor):
    """Calcula el IoU (Intersection over Union) promedio entre cajas verdaderas y predichas."""
    pairwise_iou = torchvision.ops.box_iou(y_true.squeeze(), y_pred.squeeze())
    result = torch.trace(pairwise_iou) / pairwise_iou.size()[0]
    return result

def accuracy(y_true: Tensor, y_pred: Tensor) -> Tensor:
    """Accuracy para clasificación binaria con 2 logits."""
    if y_true.dim() > 1:
        y_true = y_true.squeeze(-1)
    y_true = y_true.long()
    pred = torch.argmax(y_pred, dim=-1)
    if pred.shape != y_true.shape:
        y_true = y_true.view_as(pred)
    return (pred == y_true).float().mean()

def loss_fn(y_true, y_preds, alpha=0.7):
    """Pérdida multitarea: clasificación + regresión."""
    logits = y_preds['class_id']
    N, K = logits.shape[0], logits.shape[1]
    if K > 1:
        y = y_true['class_id']
        if y.dim() == 2 and y.size(1) == 1:
            y = y.squeeze(1)
        if y.dim() == 2 and y.size(1) == K:
            y = y.argmax(1)
        y = y.long()
        class_weights = torch.tensor([0.35, 0.65], dtype=torch.float32).to(logits.device)
        cls_loss = F.cross_entropy(logits, y, weight=class_weights)
    else:
        y = y_true['class_id']
        if y.dim() == 1:
            y = y.unsqueeze(1)
        y = y.float()
        if y.shape != logits.shape:
            y = y.view_as(logits)
        bce = torch.nn.BCEWithLogitsLoss()
        cls_loss = bce(logits, y)
    reg_pred = y_preds['bbox'].float()
    reg_true = y_true['bbox'].float()
    if reg_pred.shape != reg_true.shape:
        reg_true = reg_true.view_as(reg_pred)
    reg_loss = F.mse_loss(reg_pred, reg_true)
    total = (1 - alpha) * cls_loss + alpha * reg_loss
    return {'loss': total, 'cls_loss': cls_loss, 'reg_loss': reg_loss}

# ===========================
# ENTRENAMIENTO Y EVALUACIÓN
# ===========================

def printer(logs: ty.Dict[str, ty.Any]):
    """Callback de logging: imprime cada 10 iteraciones."""
    if logs['iters'] % 10 != 0:
        return
    print('Iteration #: ', logs['iters'])
    for name, value in logs.items():
        if name == 'iters':
            continue
        if type(value) in [float, int]:
            value = round(value, 4)
        elif type(value) is torch.Tensor:
            value = torch.round(value, decimals=4)
        print(f'\t{name} = {value}')
    print()

def evaluate(
    logs: ty.Dict[str, ty.Any],
    labels: ty.Dict[str, Tensor],
    preds: ty.Dict[str, Tensor],
    eval_set: str,
    metrics: ty.Dict[str, ty.Callable[[Tensor, Tensor], Tensor]],
    losses: ty.Optional[ty.Dict[str, Tensor]] = None,
) -> ty.Dict[str, ty.Any]:
    """Callback de evaluación (uso dentro del training loop)."""
    if losses is not None:
        for loss_name, loss_value in losses.items():
            logs[f'{eval_set}_{loss_name}'] = loss_value
    for task_name, label in labels.items():
        for metric_name, metric in metrics[task_name]:
            value = metric(label, preds[task_name])
            logs[f'{eval_set}_{metric_name}'] = value
    return logs

def step(
    model: Model,
    optimizer: Optimizer,
    batch: maskDataset,
    loss_fn: ty.Callable[[ty.Dict[str, torch.Tensor]], torch.Tensor],
    device: str,
    train: bool = False,
) -> ty.Tuple[ty.Dict[str, Tensor], ty.Dict[str, Tensor]]:
    """Un paso (train o eval): forward, loss, backward, optimizer.step()."""
    if train:
        optimizer.zero_grad()
    img = batch.pop('image').to(device)
    for k in list(batch.keys()):
        batch[k] = batch[k].to(device)
    preds = model(img.float())
    losses = loss_fn(batch, preds)
    final_loss = losses['loss']
    if train:
        final_loss.backward()
        optimizer.step()
    return losses, preds

def train(
    model: Model,
    optimizer: Optimizer,
    dataset: DataLoader,
    eval_datasets: ty.List[ty.Tuple[str, DataLoader]],
    loss_fn: ty.Callable[[ty.Dict[str, torch.Tensor]], torch.Tensor],
    metrics: ty.Dict[str, ty.Callable[[Tensor, Tensor], Tensor]],
    callbacks: ty.List[ty.Callable[[ty.Dict[ty.Any, ty.Any]], None]],
    device: str,
    train_steps: 100,
    eval_steps: 10,
) -> Model:
    """Training loop genérico (clasificación binaria + regresión de bbox)."""
    model = model.to(device)
    iters = 0
    iterator = iter(dataset)
    assert train_steps > eval_steps, 'Train steps should be greater than the eval steps'
    while iters <= train_steps:
        logs = dict()
        logs['iters'] = iters
        try:
            batch = next(iterator)
        except StopIteration:
            iterator = iter(dataset)
            batch = next(iterator)
        losses, preds = step(model, optimizer, batch, loss_fn, device, train=True)
        logs = evaluate(logs, batch, preds, 'train', metrics, losses)
        if iters % eval_steps == 0:
            model.eval()
            with torch.no_grad():
                for name, dataset in eval_datasets:
                    for batch in dataset:
                        losses, preds = step(model, optimizer, batch, loss_fn, device, train=False)
                        logs = evaluate(logs, batch, preds, name, metrics, losses)
            model.train()
        for callback in callbacks:
            callback(logs)
        iters += 1
    return model


## **Preparación del Dataset**

-   **Configuración de device**: Selección automática de GPU si está disponible.
-   **Carga y preprocesamiento**: Lectura del CSV de anotaciones, mapeo de clases a IDs, selección de columnas relevantes.
-   **Análisis exploratorio**: Estadísticas de tamaños, canales y formas de las imágenes; verificación de balance de clases y validez de bounding boxes.
-   **Normalización de bounding boxes**: Las coordenadas se escalan a [0,1] para facilitar el entrenamiento.

In [51]:
# ===========================
# Configuración básica en PyTorch
# ===========================

torch.manual_seed(32)
# Fija la semilla para que los resultados sean reproducibles.

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Usando {device}')
# Selecciona GPU si está disponible, de lo contrario usa CPU.

test = torch.ones((100, 100)).to(device)
# Crea un tensor de prueba en el dispositivo seleccionado (CPU o GPU).

del test
# Elimina el tensor de la memoria.

torch.cuda.empty_cache()
# Limpia la memoria de la GPU, dejándola lista para entrenar modelos.


Usando cuda


In [52]:
# ===========================
# CONFIGURACIÓN DE DIRECTORIOS Y PARÁMETROS
# ===========================

DATA_DIR = '/kaggle/input/aa-iv-2025-ii-object-localization'  # Carpeta donde Kaggle guarda el dataset (solo lectura)
WORK_DIR = 'workdir'                                          # Carpeta de trabajo (aquí se guardan outputs y resultados)
BATCH_SIZE = 32                                               # Tamaño de lote (batch) para el entrenamiento del modelo

# Ruta donde están guardadas las imágenes
img_dir = osp.join(DATA_DIR, "images")

# ===========================
# CARGA DEL DATASET
# ===========================

# Leemos el archivo CSV de entrenamiento que contiene:
# - nombre de la imagen
# - coordenadas de la caja delimitadora (bounding box: xmin, ymin, xmax, ymax)
# - clase del objeto
df = pd.read_csv(osp.join(DATA_DIR, "train.csv"))

# ===========================
# MAPEO DE CLASES A IDs
# ===========================

# Diccionario para convertir las clases (texto) en identificadores numéricos
# Diccionario que asigna un número a cada clase:
# - "no-mask" → 0
# - "mask"    → 1
obj2id  = {'no-mask':0,'mask':1}

# Diccionario inverso: convertir IDs numéricos en nombres de clases
id2obj  = {0:'no-mask',1:'mask'}

# Crear nueva columna "class_id" en el DataFrame con el valor numérico de la clase
df["class_id"] = df["class"].map(obj2id)

# ===========================
# SELECCIÓN DE COLUMNAS ÚTILES
# ===========================

# Definimos qué columnas necesitamos realmente del dataset
columns_f=['filename','xmin','ymin','xmax','ymax','class','class_id']

# Nos quedamos únicamente con esas columnas
df = df[columns_f].copy()


# Exploremos un poco los datos

In [53]:
df


,filename,xmin,ymin,xmax,ymax,class,class_id
0,video_CDC-YOUTUBE_mp4-63_jpg.rf.2f4f64f6ef712f...,315,249,468,374,no-mask,0
1,IMG_4860_mp4-36_jpg.rf.01a053cabddff2cdd19f04e...,257,237,299,264,no-mask,0
2,IMG_1491_mp4-12_jpg.rf.9df64033aebef44b8bb9a6a...,291,245,582,449,mask,1
3,IMG_4861_mp4-64_jpg.rf.74ab6d1da8a1fa9b8fbb576...,231,229,577,420,no-mask,0
4,IMG_9950-1-_mp4-83_jpg.rf.74dca33810c23ba144d8...,107,168,515,469,no-mask,0
...,...,...,...,...,...,...,...
214,videoplayback-1-_mp4-58_jpg.rf.bfdf3258d74f87d...,408,168,465,212,no-mask,0
215,video_CDC-YOUTUBE_mp4-36_jpg.rf.5d17748e659665...,181,232,350,356,mask,1
216,IMG_4861_mp4-38_jpg.rf.880a11c3ebf59b3d0cf988f...,112,179,413,438,no-mask,0
217,How-to-Properly-Wear-a-Face-Mask-_-UC-San-Dieg...,268,134,382,422,no-mask,0


In [54]:
# ===========================
# CARGA DE UNA IMAGEN DE EJEMPLO
# ===========================

# Construimos la ruta completa a un archivo de imagen específico.
img_filename = osp.join(DATA_DIR, "images", 'IMG_1493_mp4-21_jpg.rf.c5a3e237451e64e0674d5b0a6d556c25.jpg')

# 1) Lectura de la imagen con OpenCV (cv2)
# ----------------------------------------
# OpenCV lee las imágenes en formato BGR (Blue, Green, Red) por defecto.
img1 = cv2.imread(img_filename)

# Convertimos de BGR a RGB para que los colores sean correctos al visualizar con matplotlib.
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)

# 2) Lectura de la misma imagen con skimage (io.imread)
# -----------------------------------------------------
# La función `io.imread` de scikit-image lee las imágenes directamente en formato RGB,
# por lo tanto no es necesario hacer la conversión de colores.
img2 = io.imread(img_filename)


In [55]:
# Mostramos la forma original de la imagen: (alto, ancho, canales) → (H, W, C)
print(img1.shape)

# Transponemos para pasar a formato (canales, alto, ancho) → (C, H, W),
# que es el requerido por PyTorch. -> como se vió en clase, pytorch trabaja
# con Channel first, no Channel last.
print(img1.transpose((2,0,1)).shape)


(640, 640, 3)
(3, 640, 640)


In [56]:
# ===========================
# ANÁLISIS EXPLORATORIO DE IMÁGENES
# ===========================

# Obtenemos la lista de nombres de archivos de las imágenes desde el DataFrame
list_image = list(df.filename)

# Inicializamos listas vacías donde guardaremos información de cada imagen
data_shape = []   # Guardará la forma completa de la imagen (alto, ancho, canales)
data_dim = []     # Guardará el número de dimensiones (ej: 2 para escala de grises, 3 para RGB)
data_w = []       # Guardará el ancho de la imagen
data_h = []       # Guardará la altura de la imagen

# Recorremos todas las imágenes con una barra de progreso (tqdm)
for i in tqdm(list_image):  # Puede tardar unos ~40 segundos en recorrer todo
    # Construimos la ruta completa de la imagen
    ruta_imagen = osp.join(img_dir, i)

    # Leemos la imagen con skimage → obtenemos forma y número de dimensiones
    imagen = io.imread(ruta_imagen)
    shapes = imagen.shape      # Ejemplo: (300, 400, 3)
    dimen = imagen.ndim        # Ejemplo: 3 si es RGB, 2 si es escala de grises

    # Leemos la imagen con PIL → obtenemos ancho y alto
    imagen = Image.open(ruta_imagen)
    w, h = imagen.size         # size devuelve (ancho, alto)

    # Guardamos toda la información en las listas
    data_w.append(w)
    data_h.append(h)
    data_shape.append(shapes)
    data_dim.append(dimen)

# Construimos un DataFrame con toda la información recopilada
data_w_h = pd.DataFrame(
    [list_image, data_shape, data_dim, data_w, data_h]
).T.rename(columns={0:'filename', 1:'shapes', 2:'ndim', 3:'w', 4:'h'})


  0%|          | 0/219 [00:00<?, ?it/s]

In [57]:
# Contamos cuántas veces aparece cada forma (alto, ancho, canales) en el dataset.
data_w_h['shapes'].value_counts()

shapes
(640, 640, 3)    219
Name: count, dtype: int64

In [58]:
# Contamos cuántas veces aparece cada clase en el dataset (en formato de texto).
# Esto muestra la distribución de imágenes entre las clases "mask" y "no-mask".
df['class'].value_counts()

class
no-mask    135
mask        84
Name: count, dtype: int64

In [59]:
# Verificamos si existen errores en las coordenadas de las cajas delimitadoras (bounding boxes).

# Caso 1: xmin >= xmax
# Esto indicaría que el lado izquierdo de la caja está a la derecha del lado derecho → caja inválida.

# Caso 2: ymin >= ymax
# Esto indicaría que la parte superior de la caja está por debajo de la parte inferior → caja inválida.
df[df['xmin']>=df['xmax']].shape, df[df['ymin']>=df['ymax']].shape

((0, 7), (0, 7))

# Normalizamos los bounding box

In [60]:
# Mostramos estadísticas básicas (min, max, promedio, etc.)
# de las coordenadas de las bounding boxes: ymin, ymax, xmin, xmax.
# Sirve para verificar que las cajas estén dentro de los rangos esperados
# y detectar valores anómalos en las anotaciones.
print(df[["ymin", "ymax", "xmin", "xmax"]].describe())

             ymin        ymax        xmin        xmax
count  219.000000  219.000000  219.000000  219.000000
mean   175.296804  371.278539  217.068493  446.849315
std     67.509690   97.632620  108.656136  104.015128
min      3.000000  145.000000    0.000000  146.000000
25%    134.000000  333.000000  154.000000  382.500000
50%    168.000000  394.000000  204.000000  466.000000
75%    224.000000  431.500000  274.500000  513.000000
max    420.000000  640.000000  557.000000  640.000000


In [61]:
# ===========================
# NORMALIZACIÓN DE COORDENADAS DE LAS BOUNDING BOXES
# ===========================

# Definimos la altura y el ancho reales de las imágenes del dataset.
# En este caso todas son de 640 x 640 píxeles.
h_real = 640
w_real = 640

# Normalizamos las coordenadas de las cajas delimitadoras dividiéndolas
# entre la altura o el ancho correspondiente.
# De esta forma los valores quedan entre 0 y 1, lo que facilita el entrenamiento.
df[["ymin", "ymax"]] = df[["ymin", "ymax"]].div(h_real, axis=0)
df[["xmin", "xmax"]] = df[["xmin", "xmax"]].div(w_real, axis=0)

In [62]:
# Estadisticos normalizados
print(df[["ymin", "ymax", "xmin", "xmax"]].describe())

             ymin        ymax        xmin        xmax
count  219.000000  219.000000  219.000000  219.000000
mean     0.273901    0.580123    0.339170    0.698202
std      0.105484    0.152551    0.169775    0.162524
min      0.004687    0.226562    0.000000    0.228125
25%      0.209375    0.520312    0.240625    0.597656
50%      0.262500    0.615625    0.318750    0.728125
75%      0.350000    0.674219    0.428906    0.801562
max      0.656250    1.000000    0.870313    1.000000


In [63]:
# Ahora visualizamos el df con los bbox normalizados
df

,filename,xmin,ymin,xmax,ymax,class,class_id
0,video_CDC-YOUTUBE_mp4-63_jpg.rf.2f4f64f6ef712f...,0.492188,0.389062,0.731250,0.584375,no-mask,0
1,IMG_4860_mp4-36_jpg.rf.01a053cabddff2cdd19f04e...,0.401562,0.370312,0.467187,0.412500,no-mask,0
2,IMG_1491_mp4-12_jpg.rf.9df64033aebef44b8bb9a6a...,0.454688,0.382812,0.909375,0.701562,mask,1
3,IMG_4861_mp4-64_jpg.rf.74ab6d1da8a1fa9b8fbb576...,0.360938,0.357812,0.901563,0.656250,no-mask,0
4,IMG_9950-1-_mp4-83_jpg.rf.74dca33810c23ba144d8...,0.167187,0.262500,0.804688,0.732812,no-mask,0
...,...,...,...,...,...,...,...
214,videoplayback-1-_mp4-58_jpg.rf.bfdf3258d74f87d...,0.637500,0.262500,0.726562,0.331250,no-mask,0
215,video_CDC-YOUTUBE_mp4-36_jpg.rf.5d17748e659665...,0.282813,0.362500,0.546875,0.556250,mask,1
216,IMG_4861_mp4-38_jpg.rf.880a11c3ebf59b3d0cf988f...,0.175000,0.279687,0.645312,0.684375,no-mask,0
217,How-to-Properly-Wear-a-Face-Mask-_-UC-San-Dieg...,0.418750,0.209375,0.596875,0.659375,no-mask,0


In [64]:
# ===========================
# PARTICIÓN ENTRENAMIENTO / VALIDACIÓN (estratificada)
# ===========================
# Dividimos el DataFrame 'df' en dos subconjuntos:
#  - train_df: datos para entrenar el modelo (75%)
#  - val_df:   datos para validar el modelo (25%)
#
# Parámetros clave:
#  - stratify=df['class_id']  → mantiene la misma proporción de clases en
#    train y val (muy importante si el dataset está desbalanceado).
#  - test_size=0.25           → 25% de los datos va a validación.
#  - random_state=42           → semilla para reproducibilidad del split.
train_df, val_df = train_test_split(
    df, stratify=df['class_id'], test_size=0.25, random_state=42
)

# Tamaños resultantes de cada partición
print(train_df.shape)
print(val_df.shape)

(164, 7)
(55, 7)


**Importante**: El set de entrenamiento debe tener información acerca de la clase y las coordenadas correspondientes a los bbox

In [65]:
# ===========================
# DISTRIBUCIÓN DE CLASES EN TRAIN (en %)
# ===========================
# ahora verificamos que la distribución de las clases se mantengan en el train
# Útil para verificar que el split estratificado mantuvo el balance de clases.
train_df['class'].value_counts(normalize=True) * 100

class
no-mask    61.585366
mask       38.414634
Name: proportion, dtype: float64

In [66]:
h, w, c = 256, 256, 3

In [67]:
# ===========================
# VISUALIZACIÓN DE MUESTRAS CON CAJAS Y CLASES
# ===========================

# Carpeta raíz donde están las imágenes del split de entrenamiento
train_root_dir = osp.join(DATA_DIR, "images")#, "train"

# Instanciamos el Dataset con el DataFrame de train y forzamos tamaño de salida (w, h)
train_ds = maskDataset(train_df, root_dir=train_root_dir,output_size=(w,h))

# Número de imágenes a mostrar y desde qué índice empezar
num_imgs = 6
start_idx = 0

# Tomamos 'num_imgs' muestras consecutivas a partir de 'start_idx'
samples = [train_ds[i] for i in range(start_idx, num_imgs)]

# Extraemos por separado las imágenes, bboxes y clases de cada sample
imgs = [s['image'] for s in samples]
# Convertimos las cajas normalizadas [0,1] a píxeles con (w,h) de salida
bboxes = [normalize_bbox(s['bbox'].squeeze(),h,w) for s in samples]
classes = [s['class_id'] for s in samples]

# Dibujamos predicciones: cajas + etiquetas
# - colors: lista con un color (BGR) que se reutiliza para todas las imágenes
# - origin: punto (x,y) para el texto (10% del ancho y alto)
# - thickness y fontScale: grosor de línea y tamaño de fuente
imgs = draw_predictions(imgs, classes, bboxes, [(0, 150, 0)], (int(w*0.1), int(h*0.1)),thickness = 1,fontScale=1)#(150, 10)

# Creamos una figura grande y colocamos cada imagen en una subgráfica
fig = plt.figure(figsize=(30, num_imgs))

for i, img in enumerate(imgs):
    fig.add_subplot(1, num_imgs, i+1)
    plt.imshow(img)

# Mostramos el collage de imágenes con sus cajas y clases
plt.show()

# Transfer Learning RESNET50_2


Utilizar  **ResNet50_2**  (Wide ResNet-50-2) para localizar personas con tapabocas en imágenes es una excelente opción por varias razones técnicas:

-   **Arquitectura Profunda y Robusta**: ResNet50_2 es una red residual profunda con más filtros por capa que la ResNet50 estándar, lo que le permite aprender representaciones más ricas y complejas de las imágenes.
    
    
-   **Reconocimiento de Patrones Faciales**: Las capas profundas pueden identificar patrones faciales y distinguir entre personas con y sin tapabocas, incluso en condiciones de iluminación o poses variadas.
    


In [68]:
from torchvision.models import wide_resnet50_2, Wide_ResNet50_2_Weights

# modelo Wide ResNet50-2
weights_resnet50_2 = Wide_ResNet50_2_Weights.IMAGENET1K_V2
model_resnet50_2 = wide_resnet50_2(weights=weights_resnet50_2)

In [69]:
# Instantiate the ResNet feature extractor
pretrained_model_resnet = ResNetFeatureExtractor(model_resnet50_2).to(device)
pretrained_model_resnet.eval()

ResNetFeatureExtractor(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
    

In [70]:
summary(pretrained_model_resnet, (3, 640, 640))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 320, 320]           9,408
       BatchNorm2d-2         [-1, 64, 320, 320]             128
              ReLU-3         [-1, 64, 320, 320]               0
         MaxPool2d-4         [-1, 64, 160, 160]               0
            Conv2d-5        [-1, 128, 160, 160]           8,192
       BatchNorm2d-6        [-1, 128, 160, 160]             256
              ReLU-7        [-1, 128, 160, 160]               0
            Conv2d-8        [-1, 128, 160, 160]         147,456
       BatchNorm2d-9        [-1, 128, 160, 160]             256
             ReLU-10        [-1, 128, 160, 160]               0
           Conv2d-11        [-1, 256, 160, 160]          32,768
      BatchNorm2d-12        [-1, 256, 160, 160]             512
           Conv2d-13        [-1, 256, 160, 160]          16,384
      BatchNorm2d-14        [-1, 256, 1

# Normalización de imagen

In [71]:
# ===========================
# CÁLCULO DE MEDIA Y DESVIACIÓN ESTÁNDAR (por canal) DEL DATASET
# ===========================
# Objetivo: estimar las estadísticas de color (mean y std de R, G, B) para
# usarlas luego en una normalización tipo torchvision.transforms.Normalize(mean, std).

train_ds = maskDataset(train_df, root_dir=train_root_dir,output_size=(w,h))

# Acumuladores para medias/STD por canal (R,G,B)
means = np.zeros(3)
stds = np.zeros(3)
n_images = 0

# Recorremos todas las imágenes del split de entrenamiento
for x in train_ds:
    img = x['image']  # Imagen en formato HxWxC (RGB).
                      # (Si la imagen estuviera en uint8 [0..255], las medias/STD saldrán en esa escala.)
                      # .astype(np.float32) comentado: convertir a float puede ser útil para mayor precisión.
    n_images += 1

    # Para cada canal (0=R, 1=G, 2=B), calculamos la media y la STD de la imagen actual
    for channel in range(3):
        channel_pixels = img[..., channel]  # Todos los píxeles del canal
        # Se acumula la media y la desviación estándar por imagen (promedio de medias, no ponderado por píxeles)
        means[channel] += np.mean(channel_pixels)
        stds[channel] += np.std(channel_pixels)

# Promediamos sobre el número de imágenes para obtener la estimación final por canal
means /= n_images
stds /= n_images



In [72]:
# INSPECCIÓN DE ESTADÍSTICAS POR CANAL
# ===========================
# 'means': medias por canal [R, G, B] calculadas en el bloque anterior.
# 'stds' : desviaciones estándar por canal [R, G, B].
# Útil para configurar transforms.Normalize(mean, std).
print(means)
print(stds)

[150.87213377 140.8888561  133.6496836 ]
[62.79959127 61.64436314 59.85598115]


# Transformación de imagenes

Se hace uso de la librería de aumentación de imagenes en https://albumentations.ai/docs/examples/pytorch_classification/

- Se utilizo solo la transformacion de Flip Horizontal y aplicamos un flitro tipo Sephia, se probaron diferentes tipos de transformaciones pero empeoraban las metricas del modelo por lo que despues de la experimentacion exsaustiva se opto por solo esas dos transformaciones

In [73]:
# ===========================
# DATA AUGMENTATION (Albumentations) + Wrapper para el pipeline
# ===========================

train_data_augmentations = A.Compose([
    A.HorizontalFlip(p=1),    # Volteo horizontal con prob. 0.5.
    #A.Rotate(limit=90, p=1, interpolation=cv2.INTER_LINEAR, border_mode=cv2.BORDER_REFLECT_101), # Rotation by 90 degrees with 50% probability for each direction.

    ],
    bbox_params=A.BboxParams(
        format='albumentations',   # Formato de bboxes esperado por Albumentations:
                                   # [x_min, y_min, x_max, y_max] NORMALIZADO en [0,1].
        label_fields=[],           # No se pasan etiquetas (category_ids) a las transforms.
    )
)

# En nuestro pipeline, las transforms operan sobre 'sample' (dict).
# Usamos un wrapper que aplica Albumentations sobre sample['image'] y sample['bbox'].
dataaug_transforms = torchvision.transforms.Compose(
    [
        AlbumentationsWrapper(train_data_augmentations)  # Aplica A.Compose a 'image' y 'bbox'.
    ]
)

In [74]:
import shutil
import re  # Usaremos expresiones regulares para extraer números de cualquier nombre de archivo

# ============================================================
# 1) PREPARAR CARPETA DE SALIDA PARA IMÁGENES FINALES
#    (BORRAR SI EXISTE Y CREAR DE NUEVO)
# ============================================================
if os.path.exists('data_final'):
    shutil.rmtree('data_final')   # Eliminamos la carpeta anterior para empezar “limpio”

os.mkdir('data_final')            # Carpeta donde guardaremos: imágenes aumentadas + originales

# Dataset base SIN resize (conserva tamaño original).
train_ds_da = maskDataset(train_df, root_dir=train_root_dir) 

# ============================================================
# 2) CÁLCULO ROBUSTO DEL ÚLTIMO ÍNDICE A PARTIR DEL NOMBRE
#    Funciona con .jpg o .jpeg y con nombres arbitrarios.
#    Ej.: 'IMG_4921-2...jpg' → extrae '4921' y usa el ÚLTIMO grupo de dígitos.
# ============================================================
def extract_any_int(name: str) -> int:
    base, _ = os.path.splitext(name)  # separa nombre y extensión
    nums = re.findall(r'\d+', base)   # encuentra todos los grupos de dígitos
    return int(nums[-1]) if nums else -1  # toma el último grupo si existe; si no, -1

last_index = train_ds_da.df.filename.apply(extract_any_int).max()
if last_index < 0:    # si ningún archivo tenía dígitos, empezamos desde 0
    last_index = 0
index = int(last_index) + 1   # primer índice nuevo para imágenes sintéticas

# ============================================================
# 3) GENERAR IMÁGENES AUMENTADAS Y SUS ANOTACIONES
#    - Recorremos el dataset de train
#    - Aplicamos data augmentation (p. ej., flip horizontal)
#    - Guardamos imagen aumentada
#    - Registramos fila con filename, class_id y bbox
# ============================================================
rows = []
for j in range(0,1):  # Cantidad de imágenes sintéticas por imagen original (aquí: 1)
    iterador = iter(train_ds_da)
    for i in range(len(train_ds_da)):
        x = next(iterador)                 # sample original: {'image', 'bbox', 'class_id', ...}
        x_transformed = copy.deepcopy(x)   # copiamos para no modificar el original
        x_transformed = dataaug_transforms(x_transformed)  # aplicamos augmentations

        # Construimos nombre único para la imagen aumentada
        filename = f"image_id_{index}_t{j}.jpeg"

        # Recuperamos la imagen aumentada (numpy HxWxC, RGB)
        image = x_transformed['image']  # .astype('uint8') opcional si fuera necesario

        # Guardamos en disco (cv2 usa BGR, por eso convertimos de RGB→BGR)
        cv2.imwrite("data_final/"+filename, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

        # Registramos anotación:
        #   - filename de la nueva imagen
        #   - class_id (tal como viene en el sample)
        #   - bbox (xmin, ymin, xmax, ymax)
        # Importante: aquí se asume que las bboxes se mantienen en el mismo formato
        # (p. ej., normalizadas [0,1]) que maneja el resto del pipeline.
        row = [filename, *x_transformed["class_id"], *x_transformed['bbox'].squeeze()]
        rows.append(row)
        index += 1

# Construimos DataFrame con las anotaciones de las imágenes aumentadas
aug_df = pd.DataFrame(rows, columns=['filename', 'class_id', 'xmin', 'ymin', 'xmax', 'ymax',])

# ============================================================
# 4) COPIAR TAMBIÉN LAS IMÁGENES ORIGINALES A 'data_final'
#    (tendremos en una misma carpeta originales + aumentadas)
# ============================================================
source = train_root_dir
destination = 'data_final'
allfiles = os.listdir(source)

for f in allfiles:
    if f in train_df['filename'].values:   # solo las del split de entrenamiento
        src_path = os.path.join(source, f)
        dst_path = os.path.join(destination, f)
        shutil.copy(src_path, dst_path)

# ============================================================
# 5) UNIR ANOTACIONES: ORIGINALES + AUMENTADAS
#    Y AGREGAR LA COLUMNA 'class' A PARTIR DE 'class_id'
# ============================================================
dataframe_with_dataaugmentation = pd.concat([train_df, aug_df], ignore_index=True)
dataframe_with_dataaugmentation['class'] = dataframe_with_dataaugmentation['class_id'].replace(id2obj)

# Mostramos el DataFrame final con todas las anotaciones
dataframe_with_dataaugmentation


,filename,xmin,ymin,xmax,ymax,class,class_id
0,IMG_4921-2_mp4-124_jpg.rf.60e2e62f7f6c331d5960...,0.000000,0.326562,0.501563,0.781250,no-mask,0
1,IMG_3099_mp4-26_jpg.rf.44828067865615f50965e95...,0.189062,0.250000,0.718750,0.648438,no-mask,0
2,videoplayback-1-_mp4-0_jpg.rf.2b8492685ce5a86f...,0.667188,0.298438,0.731250,0.351562,no-mask,0
3,video_CDC-YOUTUBE_mp4-31_jpg.rf.9dcb8f35940393...,0.428125,0.389062,0.598437,0.528125,mask,1
4,Apple-Tests-Face-ID-Feature-While-Wearing-a-Ma...,0.245312,0.278125,0.662500,0.553125,no-mask,0
...,...,...,...,...,...,...,...
323,image_id_5295680176929_t0.jpeg,0.365625,0.462500,0.404687,0.485938,mask,1
324,image_id_5295680176930_t0.jpeg,0.001563,0.264062,0.515625,0.710938,no-mask,0
325,image_id_5295680176931_t0.jpeg,0.145312,0.296875,0.707812,0.818750,mask,1
326,image_id_5295680176932_t0.jpeg,0.268750,0.259375,0.500000,0.562500,mask,1


In [75]:
# ===========================
# VERIFICACIÓN RÁPIDA DE FORMAS (nº de filas y columnas)
# ===========================
# Muestra un par de tuplas:
#  - Primero: shape de train_df  → (n_filas_train, n_columnas)
#  - Segundo: shape de dataframe_with_dataaugmentation → (n_filas_total, n_columnas)
#

train_df.shape, dataframe_with_dataaugmentation.shape


((164, 7), (328, 7))

In [76]:
# ===========================
# PIPELINE DE TRANSFORMACIONES
# - common_transforms: pasos comunes a train y valid/test
# - train_data_augmentations: augmentations solo para entrenamiento
# - train_transforms: augmentations + comunes (en ese orden)
# - eval_transforms: solo comunes (sin augmentations)
# ===========================

common_transforms = [
    ToTensor(),               # Convierte imagen de numpy (H,W,C) → torch.Tensor (C,H,W), float32
    Normalizer(               # Normaliza por canal: (x - mean) / std
        means=means,          
        stds=stds,            
    )
]

train_data_augmentations = A.Compose([
    A.ToSepia(p=1) # Apply sepia filter

    ],
    bbox_params=A.BboxParams(
        format='albumentations',  # Formato esperado: [xmin, ymin, xmax, ymax] NORMALIZADO en [0,1]
        label_fields=[],          
    )
)

# En entrenamiento: primero augmentations (operan sobre numpy HxWxC),
# luego ToTensor() y Normalizer() (operan sobre tensor CxHxW).
train_transforms = torchvision.transforms.Compose(
    [
        AlbumentationsWrapper(train_data_augmentations),  # Aplica A.Compose a 'image' y 'bbox'
    ] + common_transforms
)

# En validación/evaluación: NO se aplican augmentations, solo los pasos comunes
# (ToTensor + Normalizer) para mantener consistencia.
eval_transforms = torchvision.transforms.Compose(common_transforms)


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


In [77]:
# ===========================
# DATASET + DATALOADER (entrenamiento)
# ===========================
# Creamos el Dataset a partir del DataFrame que une originales + aumentadas.
# root_dir='data_final'  → carpeta donde guardamos todas las imágenes (originales y sintéticas).
# transform=train_transforms → aplica (1) augmentations (AlbumentationsWrapper) y (2) pasos comunes (ToTensor + Normalizer).
# output_size=(w,h)      → fuerza que todas las imágenes salgan con el mismo tamaño (p. ej., 640x640).
train_ds = maskDataset(dataframe_with_dataaugmentation, root_dir='data_final', transform=train_transforms,output_size=(w,h)) #train_root_dir

# DataLoader: empaqueta el dataset en lotes (batches) para entrenamiento.
# batch_size=16 → cada iteración entrega 16 muestras (imágenes + labels si labeled=True).
train_data = torch.utils.data.DataLoader(train_ds, batch_size=16)

# Iteramos una sola vez sobre el DataLoader para inspeccionar la forma del tensor de imágenes.
# Esperado en PyTorch (channel-first): [batch, channels, height, width] → (16, 3, h, w)
for x in train_data:
    print(x['image'].size())  
    break                     # 'break' para no consumir todo el DataLoader en esta comprobación


torch.Size([16, 3, 256, 256])


Nota: Se verifica que el tensor tenga forma [B,C,H,W]

In [78]:
# Libera la memoria **en caché** de CUDA que PyTorch reservó pero no está usando.
# No borra tensores activos ni reduce memoria de objetos vivos.
# Útil tras `del` de tensores grandes para evitar OOM, pero abusar puede bajar rendimiento.
torch.cuda.empty_cache()

In [79]:
# Imprime el tamaño del tensor de imagen que viene en x['image'].
print('image', x['image'].size())

# Instancia el modelo con el tamaño de entrada (3, h, w) y 2 clases.
model = Model(input_shape=(3, h, w), n_classes=2).to(device)

# Mueve el tensor de imágenes al mismo device que el modelo (cuda o cpu).
x['image'] = x['image'].to(device)

# Forward: pasa el batch de imágenes por el modelo.
# Salida esperada (diccionario):
#   - preds['bbox']: tensor [B, 4] con las coordenadas predichas (en la misma escala que las etiquetas).
#   - preds['class_id']: tensor [B, n_classes] con logits de clasificación.
preds = model(x['image'])

# Muestra el diccionario de predicciones.
preds


image torch.Size([16, 3, 256, 256])


{'bbox': tensor([[-0.3548,  0.5903,  0.8692,  0.2882],
         [-0.1962,  0.2306, -0.2824,  0.2189],
         [-0.1053,  0.1951,  0.0911,  0.3712],
         [ 0.5158,  1.2994, -0.2871, -0.0222],
         [-0.0481,  0.1208,  0.2055, -0.4125],
         [-0.1586,  0.6600, -0.0941,  0.0755],
         [ 0.0841, -0.3063,  0.4624,  0.5121],
         [-0.4052,  0.5987,  0.5357, -0.3930],
         [-0.3567,  0.5051,  0.2428,  0.3642],
         [ 0.6720,  0.6685,  0.3395,  0.6756],
         [-0.9622,  0.0955,  0.4901, -0.2025],
         [ 0.7583,  1.4584,  0.7085, -0.1757],
         [-0.7663,  0.6415,  0.4360, -0.0367],
         [-0.6181,  0.3590,  0.1921, -0.2193],
         [-0.3429,  0.2445,  0.0462,  0.3567],
         [ 0.3432,  0.1146,  0.2729,  0.0117]], device='cuda:0',
        grad_fn=<AddmmBackward0>),
 'class_id': tensor([[ 1.0195, -0.4723],
         [ 0.4905, -0.2123],
         [ 0.8049, -0.8269],
         [ 0.1396,  0.5523],
         [-0.0537,  0.3746],
         [ 0.3589, -0.4843],
 

# Bucle de entrenamiento/ training loop

# Run

In [80]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


- El batch size se definió como resultado de diferentes experimentos y comparación de resultados, con valores inferiores o superiores a 32 teniamos problemas de Memoria desbordada o las metricas se desmejoraban

In [81]:
# ===========================
# RUN: configuración rápida para lanzar entrenamiento/validación
# ===========================

# Hparams: hiperparámetros básicos del run
batch_size = 32

# Data: datasets y transforms
# - train: augmentations + ToTensor + Normalize, sobre imágenes (w,h)
# - val  : solo ToTensor + Normalize (sin augmentations)
train_ds = maskDataset(
    dataframe_with_dataaugmentation, root_dir='data_final',
    transform=train_transforms, output_size=(w,h)
)  # ,output_size=(255,255)

val_ds = maskDataset(
    val_df, root_dir=train_root_dir,
    transform=eval_transforms, output_size=(w,h)
)  # ,output_size=(255,255)

# DataLoaders: batching y paralelismo
# - shuffle solo en train
# - num_workers = cpu_count() para acelerar lectura/transform
train_data = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=cpu_count())
val_data   = DataLoader(val_ds,   batch_size=batch_size,               num_workers=cpu_count())

# Model: instancia tu arquitectura (por defecto: input (3,640,640) y 2 clases si ajustaste el Model)
# - .to(device): mueve a GPU/CPU
model = Model().to(device)
summary(model, model.input_shape)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           9,408
       BatchNorm2d-2         [-1, 64, 128, 128]             128
              ReLU-3         [-1, 64, 128, 128]               0
         MaxPool2d-4           [-1, 64, 64, 64]               0
            Conv2d-5          [-1, 128, 64, 64]           8,192
       BatchNorm2d-6          [-1, 128, 64, 64]             256
              ReLU-7          [-1, 128, 64, 64]               0
            Conv2d-8          [-1, 128, 64, 64]         147,456
       BatchNorm2d-9          [-1, 128, 64, 64]             256
             ReLU-10          [-1, 128, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]          32,768
      BatchNorm2d-12          [-1, 256, 64, 64]             512
           Conv2d-13          [-1, 256, 64, 64]          16,384
      BatchNorm2d-14          [-1, 256,

- Se separaron los learning rates de backbone, clasificación y regresión. Según la literatura y experimentos, para modelos preentrenados es recomendable usar un learning rate bajo en el backbone para evitar perder el conocimiento aprendido. El learning rate de la cabeza de clasificación se mantuvo más bajo que el de regresión, permitiendo equilibrar el ritmo de aprendizaje entre ambas tareas y evitar que la clasificación domine el entrenamiento antes de que la regresión converja, se obtuvieron mejores resultados en clasificacion con LR de 1e-4 que con 1e-5.

In [82]:
# ===========================
# OPTIMIZER + LANZAR ENTRENAMIENTO
# ===========================

# Optimizer: AdamW con learning rate 'lr' sobre TODOS los parámetros del modelo.
optimizer = torch.optim.AdamW([
    {"params": model.backbone.parameters(), "lr": 1e-5},
    {"params": model.cls_head.parameters(), "lr": 1e-4},
    {"params": model.reg_head.parameters(), "lr": 1e-2}
], weight_decay=1e-4)

# Loop de entrenamiento:
# - 'train_data' como DataLoader de entrenamiento
# - 'eval_datasets': lista de pares (nombre_split, DataLoader) para evaluación periódica
# - 'loss_fn': pérdida multi-tarea (cls + bbox)
# - 'metrics':
#     • bbox  → IoU
#     • class → accuracy (binaria con 2 logits en este proyecto)
# - 'callbacks': funciones de logging/monitoreo (p. ej., printer)
# - 'train_steps' y 'eval_steps': frecuencia de entrenamiento y evaluación
model = train(
    model,
    optimizer,
    train_data,
    eval_datasets=[('val', val_data)],
    loss_fn=loss_fn,
    metrics={
        'bbox': [('iou', iou)],
        'class_id': [('accuracy', accuracy)]
    },
    callbacks=[printer],
    device=device,
    train_steps=100,
    eval_steps=10
)


Iteration #:  0
	train_loss = 0.412200003862381
	train_cls_loss = 0.671500027179718
	train_reg_loss = 0.3010999858379364
	train_iou = 0.0066
	train_accuracy = 0.5625
	val_loss = 0.3698999881744385
	val_cls_loss = 0.6841999888420105
	val_reg_loss = 0.2353000044822693
	val_iou = 0.0049
	val_accuracy = 0.7825999855995178

Iteration #:  10
	train_loss = 0.26570001244544983
	train_cls_loss = 0.5899999737739563
	train_reg_loss = 0.1266999989748001
	train_iou = 0.0198
	train_accuracy = 0.625
	val_loss = 1.132200002670288
	val_cls_loss = 0.6376000046730042
	val_reg_loss = 1.3442000150680542
	val_iou = 0.0254
	val_accuracy = 0.9129999876022339

Iteration #:  20
	train_loss = 0.1941000074148178
	train_cls_loss = 0.6008999943733215
	train_reg_loss = 0.019700000062584877
	train_iou = 0.2369
	train_accuracy = 0.7390999794006348
	val_loss = 0.26429998874664307
	val_cls_loss = 0.5321999788284302
	val_reg_loss = 0.14949999749660492
	val_iou = 0.157
	val_accuracy = 0.9129999876022339

Iteration #:  30


# Análisis de algunos resultados (muestra).



In [83]:
num_imgs = 8
ncols = 8
nrows = math.ceil(num_imgs / ncols)  # nº de filas para la grilla de visualización

start_idx = 0

# ===========================
# 1) CONSTRUIR LOTE DE INFERENCIA (SIN TRANSFORMS)
# ===========================
# Tomamos 'num_imgs' ejemplos del split de validación (val_df) para inferencia/visualización.
# - root_dir: carpeta de imágenes originales.
# - output_size=(w,h): aseguramos tamaño uniforme (p.ej., 640x640).
# 
inference_ds = maskDataset(val_df.iloc[start_idx:start_idx+num_imgs], root_dir=train_root_dir,output_size=(w,h))

# DataLoader con batch = num_imgs para procesar todo el subconjunto de una vez (sin barajar).
inference_data = DataLoader(inference_ds, batch_size=num_imgs, num_workers=1, shuffle=False)

# Extraemos un batch (diccionario con 'image', 'bbox', 'class_id')
inference_batch = next(iter(inference_data))

# Preasignamos un arreglo donde guardaremos las imágenes YA transformadas a tensor (N, C, H, W)
inference_imgs = np.empty((num_imgs, 3, h, w))

# Usaremos las transformaciones de evaluación (ToTensor + Normalizer) definidas antes.
# Estas esperan un 'sample' con clave 'image' y devuelven 'image' como tensor CxHxW normalizado.
transform = eval_transforms

# ===========================
# 2) APLICAR TRANSFORMACIONES DE EVAL A CADA IMAGEN DEL BATCH
# ===========================
# El DataLoader devuelve 'inference_batch["image"]' como tensor (N, H, W, C) o arreglo convertible.
# Recorremos por imagen, aplicamos eval_transforms y guardamos en 'inference_imgs' con forma (C,H,W).
for i, img in enumerate(inference_batch['image']):
    # Convertimos a numpy (HxWxC) si viniera como tensor y aplicamos el wrapper de transforms
    inference_imgs[i] = transform(dict(image=img.numpy()))['image'].numpy()

# ===========================
# 3) INFERENCIA CON EL MODELO
# ===========================
# Convertimos 'inference_imgs' a tensor float en el device (cuda/cpu) y pasamos por el modelo.
preds = model(torch.tensor(inference_imgs).float().to(device))

# ===========================
# 4) PREPARAR ELEMENTOS PARA VISUALIZACIÓN (GT vs PRED)
# ===========================
# Tomamos las mismas muestras del Dataset (sin transforms) para dibujar imágenes originales.
samples = [inference_ds[i] for i in range(start_idx, num_imgs)]

# Imágenes originales (numpy HxWxC)
imgs = [s['image'] for s in samples]

# BBoxes ground-truth en píxeles para dibujar:
#  - s['bbox'] se asume normalizada [0,1]; la convertimos a píxeles con normalize_bbox(h,w).
bboxes = [normalize_bbox(s['bbox'].squeeze(), h, w) for s in samples]

# Clases ground-truth (enteros), tal como están en el sample.
classes = [s['class_id'] for s in samples]

# ===========================
# 5) POSTPROCESO DE PREDICCIONES
# ===========================
# Cajas predichas:
#  - preds['bbox'] es un tensor [N,4] en la MISMA escala que las etiquetas (normalizada si entrenaste así).
#  - Convertimos a numpy y a píxeles para dibujar.
pred_bboxes = preds['bbox'].detach().cpu().numpy()
pred_bboxes = [normalize_bbox(bbox, h, w) for bbox in pred_bboxes]

# Clases predichas (logits → argmax). Resultado: ids de clase por imagen.
pred_classes = preds['class_id'].argmax(-1).detach().cpu().numpy()


In [ ]:
# ===========================
# VISUALIZACIÓN: GT (verde) vs PRED (rojo) — versión robusta
# ===========================

# Determinar cuántos ejemplos hay realmente en cada lista/salida
n = min(len(imgs), len(bboxes), len(pred_bboxes), len(pred_classes))

# --- GT en VERDE ---
imgs = draw_predictions(
    imgs[:n], classes[:n], bboxes[:n],
    [(0, 150, 0)], (int(w*0.1), int(h*0.1)),
    thickness=1, fontScale=1
)

# --- PRED en ROJO ---
# Adaptar clases predichas al formato esperado por draw_predictions
pred_classes_ = [np.array([c]) for c in pred_classes[:n]]

imgs = draw_predictions(
    imgs[:n], pred_classes_, pred_bboxes[:n],
    [(200, 0, 0)], (int(w*0.8), int(h*0.8)),
    thickness=1, fontScale=1
)

# --- GRID de visualización ---
# Recalcular filas/columnas en función de n
ncols_eff = min(ncols, n)             # ncols original si cabe; si no, recorta
nrows_eff = math.ceil(n / ncols_eff)

fig, axes = plt.subplots(nrows=nrows_eff, ncols=ncols_eff, figsize=(30, 30))

# Asegurar un iterable 1D de ejes
axes_flat = np.array(axes).reshape(-1) if isinstance(axes, np.ndarray) else np.array([axes])

for i in range(n):
    axes_flat[i].imshow(imgs[i])
    axes_flat[i].axis('off')

# Ocultar ejes sobrantes si la grilla es más grande que n
for j in range(n, len(axes_flat)):
    axes_flat[j].axis('off')

plt.tight_layout()
plt.show()

In [85]:
# Guarda el **modelo completo** (arquitectura + pesos) en disco.
torch.save(model, 'pretrained_model.pth')


# Submission

In [86]:
# Detectar dispositivo
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Usando: {device}')
model = model.to(device)
model.eval()  # modo inferencia

# Rutas y datos de test
test_root_dir = osp.join(DATA_DIR, "images")
test_df = pd.read_csv(osp.join(DATA_DIR, "test.csv"))

# Dataset de test (usa tu clase correcta: maskDataset)
test_ds = maskDataset(
    test_df,
    root_dir=test_root_dir,
    labeled=False,
    transform=eval_transforms,
    output_size=(w, h)
)

# DataLoader de test
test_data = DataLoader(test_ds, batch_size=1, num_workers=cpu_count(), shuffle=False)

# Listas de salida
class_preds, bbox_preds = [], []

# Bucle de inferencia
with torch.no_grad():
    for batch in test_data:
        imgs = batch['image'].float().to(device)
        out = model(imgs)

        # Predicciones
        class_pred = out['class_id'].argmax(dim=-1).detach().cpu().numpy()
        bbox_pred = out['bbox'].detach().cpu().numpy()

        # Guardar
        class_preds.append(class_pred.squeeze())
        bbox_preds.append(bbox_pred.squeeze())

Usando: cuda


In [87]:
# Convertir las listas de predicciones en arreglos de NumPy
# Esto facilita operaciones vectorizadas y el posterior guardado en archivo de submission
class_preds = np.array(class_preds)   # Arreglo con las clases predichas (una por imagen)
bbox_preds = np.array(bbox_preds)     # Arreglo con las cajas predichas (coordenadas por imagen)

In [88]:
submission = pd.DataFrame(
    index=test_df.filename,   # Usar los nombres de archivo del conjunto de test como índice
    data={
        'class_id': class_preds,  # Columna con las clases predichas para cada imagen
        }
)

In [89]:
submission["xmin"] = bbox_preds[:, 0]*w_real  # Coordenada X mínima de la caja, escalada al ancho real de la imagen
submission["ymin"] = bbox_preds[:, 1]*h_real  # Coordenada Y mínima de la caja, escalada a la altura real de la imagen
submission["xmax"] = bbox_preds[:, 2]*w_real  # Coordenada X máxima de la caja, escalada al ancho real de la imagen
submission["ymax"] = bbox_preds[:, 3]*h_real  # Coordenada Y máxima de la caja, escalada a la altura real de la imagen


In [90]:
submission['class'] = submission['class_id'].replace(id2obj)  # Reemplaza los IDs de clase numéricos por sus nombres/etiquetas reales usando el diccionario id2obj


In [91]:
submission['class'].value_counts()  # Muestra la cantidad de predicciones por cada clase (frecuencia de cada etiqueta en el submission)


class
no-mask    28
mask       27
Name: count, dtype: int64

In [92]:
submission.to_csv('submission_resnet.csv')  # Exporta el DataFrame de submission a un archivo CSV con el nombre 'submission_vgg16.csv'
